In [16]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data and preparing DataFrames

#### MID

In [17]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
#print(mid.describe())
# #bring the time data into a new df:
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
masses = [2,4,12,15,16,18,28,32,40,44]

#add elapsed time columns
def elapsed_time(df):
    counter = len(df.columns)
    for idx, col in enumerate(df):
        timestamp = pd.to_datetime(df[col], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp - timestamp[0]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[idx]) 
        df.insert(counter, colname, runtime)
        counter += 1
    return df
print(elapsed_time(time))
#print(time)
#print(elapsed_time)
time_sliced = time.iloc[:,10:].fillna(0) #select the time columns
print(time_sliced)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[2,0],format="%Y/%m/%d %H:%M:%S.%f")
    dic= {}
    for idx, mass in enumerate(masses):
        key = "Timestamp, mass {}".format(mass)
        content = df2.iloc[:, idx].dropna().values
        print(content)
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=row*1000)))
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df.columns

               mass 2.0_time            mass 4.0_time  \
0    2023/05/08 16:41:15.599  2023/05/08 16:41:15.799   
1    2023/05/08 16:41:17.672  2023/05/08 16:41:18.273   
2    2023/05/08 16:41:22.746  2023/05/08 16:41:22.946   
3    2023/05/08 16:41:28.320  2023/05/08 16:41:28.520   
4    2023/05/08 16:41:33.895  2023/05/08 16:41:34.095   
..                       ...                      ...   
529  2023/05/08 17:30:20.770  2023/05/08 17:30:20.970   
530  2023/05/08 17:30:26.345  2023/05/08 17:30:26.545   
531  2023/05/08 17:30:31.920  2023/05/08 17:30:32.120   
532  2023/05/08 17:30:37.495  2023/05/08 17:30:37.695   
533  2023/05/08 17:30:43.070  2023/05/08 17:30:43.270   

              mass 12.0_time           mass 15.0_time  \
0    2023/05/08 16:41:15.999  2023/05/08 16:41:16.200   
1    2023/05/08 16:41:18.673  2023/05/08 16:41:18.873   
2    2023/05/08 16:41:23.146  2023/05/08 16:41:23.346   
3    2023/05/08 16:41:28.720  2023/05/08 16:41:28.920   
4    2023/05/08 16:41:34.295  

Index(['Timestamp, mass 2', 'Timestamp, mass 4', 'Timestamp, mass 12',
       'Timestamp, mass 15', 'Timestamp, mass 16', 'Timestamp, mass 18',
       'Timestamp, mass 28', 'Timestamp, mass 32', 'Timestamp, mass 40',
       'Timestamp, mass 44'],
      dtype='object')

#### Vaclogger

In [18]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,100mbar inj,Barion_2,Barion_1,DUAL experiment,DUAL insulation,1mbar inj,helium,T-platinum,T-CERNOX,V_heater,DUAL inj line,I_emission,I_grid
0,NaN,2023-05-08 16:15:06,0.0,-0.021887,2.610000e-09,1.040000e-09,4.996000e-09,5.363000e-07,-0.000730,0.0,69.921,84.287,0.000015,0.000006,NaN,NaN
1,NaN,2023-05-08 16:15:14,8.0,-0.021788,2.630000e-09,1.040000e-09,4.996000e-09,5.331000e-07,-0.000751,0.0,69.697,84.960,0.000015,0.000006,NaN,NaN
2,NaN,2023-05-08 16:15:23,17.0,-0.021663,2.630000e-09,1.030000e-09,4.996000e-09,5.204000e-07,-0.000793,0.0,69.023,85.634,0.000017,0.000006,NaN,NaN
3,NaN,2023-05-08 16:15:31,25.0,-0.021318,2.630000e-09,1.040000e-09,4.996000e-09,4.966000e-07,-0.000770,0.0,68.799,86.083,0.000018,0.000006,NaN,NaN
4,NaN,2023-05-08 16:15:40,34.0,-0.021961,2.620000e-09,1.030000e-09,4.996000e-09,4.722000e-07,-0.000788,0.0,68.125,86.868,0.000018,0.000006,NaN,NaN


In [19]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")

#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["I_em"]=[element * 1e-6 for element in hv["IMon2"]]

# hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["I_em"]=abs(hv["I_em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,Unnamed: 19,hv_grid,I_em
0,2023-05-08 16:41:14,08/05/2023,0.000,16:41:14.295,0.0,0.06,0.04,0.08,0.0,0.0,...,0.0083,0.0061,0.0150,0.009,0.0045,-0.0145,NaN,NaN,7.200000e-09,6.600000e-09
1,2023-05-08 16:41:15,08/05/2023,1.189,16:41:15.484,0.0,0.06,0.04,0.08,0.0,0.0,...,0.0083,0.0061,0.0143,0.009,0.0045,-0.0145,NaN,NaN,7.200000e-09,6.600000e-09
2,2023-05-08 16:41:16,08/05/2023,2.355,16:41:16.650,0.0,0.06,0.04,0.08,0.0,0.0,...,0.0083,0.0061,0.0143,0.009,0.0045,-0.0145,NaN,NaN,7.200000e-09,6.600000e-09
3,2023-05-08 16:41:17,08/05/2023,3.539,16:41:17.834,0.0,0.06,0.04,0.08,0.0,0.0,...,0.0084,0.0061,0.0143,0.009,0.0045,-0.0145,NaN,NaN,7.200000e-09,6.600000e-09
4,2023-05-08 16:41:19,08/05/2023,4.718,16:41:19.013,0.0,0.06,0.04,0.08,0.0,0.0,...,0.0084,0.0061,0.0143,0.009,0.0045,-0.0145,NaN,NaN,7.200000e-09,6.700000e-09


## Data processing

#### Comments

In [20]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'LHe transfer complete' 'starting ESD measurement']


In [21]:
#print hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv.loc[:,["datetime","Comment"]]
hv_comm.dropna(inplace=True)
hv_comm.index

[nan 'grid repelling' 'grid transparent' 'grid repelling 2'
 'grid transparent 2' 'grid repelling 3' 'grid transparent 3'
 'grid repelling 4' 'grid transparent 4' 'grid repelling 5'
 'emission off ']


Int64Index([ 225,  226,  227,  228,  565,  566,  567,  568,  947,  948,  949,
             950,  951, 1367, 1368, 1369, 1370, 1454, 1455, 1456, 1457, 1886,
            1887, 1888, 1889, 1981, 1982, 1983, 1984, 2180, 2181, 2182, 2183,
            2266, 2267, 2268, 2269, 2270, 2277, 2278, 2279, 2280, 2284, 2285,
            2286, 2287],
           dtype='int64')

#### CernOx temperature R-T conversion

In [22]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Plotting

In [23]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [24]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(new_df["Timestamp, mass 4"],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(new_df["Timestamp, mass 12"],mid["mass 12.0_value"],marker=".", markersize=4,label='M12')
plt.plot(new_df["Timestamp, mass 15"],mid["mass 15.0_value"],marker=".", markersize=4,label='M16')
plt.plot(new_df["Timestamp, mass 16"],mid["mass 16.0_value"],marker=".", markersize=4,label='M15')
plt.plot(new_df["Timestamp, mass 18"],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(new_df["Timestamp, mass 28"],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(new_df["Timestamp, mass 32"],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(new_df["Timestamp, mass 40"],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(new_df["Timestamp, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')
plt.xlabel('Timestamp')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S2 CU ESD @4K EGA halfway - MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
""" 
#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.95,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)

 """
plt.savefig(r'./graphs/MID.png')
plt.show()


In [ ]:
#plot the h2 mid 
plt.figure(figsize=(12,6))
plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')

plt.xlabel('Timestamp')
plt.ylabel('Ion current (A)')
plt.legend(loc="upper right")
plt.title("S2 CU ESD @4K EGA halfway - MID of H2")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('linear')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

plt.savefig(r'./graphs/H2 MID.png')
plt.show()


In [ ]:
#plotting HiVolta log
plt.figure(figsize=(12,6))

plt.plot(hv["datetime"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["datetime"],hv["I_em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Timestamp')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("S2 CU ESD @4K, EGA halfway - HiVolta log")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.8,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/HiVolta.png')
plt.show()


In [ ]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
#plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S2 CU ESD @4K, EGA halfway - Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/Pressures.png')
plt.show()

In [ ]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S2 CU ESD @4K, EGA halfway - Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()